In [1]:
from transformers import BertForMaskedLM, BertTokenizer, AutoModelForMaskedLM, AutoTokenizer, pipeline

# Zadanie 2

Pobranie 3 polskich modeli

In [2]:
model_dkleczek = BertForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")
tokenizer_dkleczek = BertTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")
pipeline_dkleczek = pipeline("fill-mask", model=model_dkleczek, tokenizer=tokenizer_dkleczek)

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model_geotrend = BertForMaskedLM.from_pretrained("Geotrend/bert-base-pl-cased")
tokenizer_geotrend = BertTokenizer.from_pretrained("Geotrend/bert-base-pl-cased")
pipeline_geotrend = pipeline("fill-mask", model=model_geotrend, tokenizer=tokenizer_geotrend)

In [4]:
model_herbert = AutoModelForMaskedLM.from_pretrained("allegro/herbert-large-cased")
tokenizer_herbert = AutoTokenizer.from_pretrained("allegro/herbert-large-cased")
pipeline_herbert = pipeline("fill-mask", model=model_herbert, tokenizer=tokenizer_herbert)

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
pipelines = [pipeline_dkleczek, pipeline_geotrend, pipeline_herbert]
models_names = ['DKleczek', 'Geotrend', 'Allegro Herbert']

# Zadanie 3

Predykcja brakujących słów w zdaniach nastawiona na poprawną odmianę przez przypadki

In [6]:
sentences = [
    'Warszawa to największe [MASK].',
    'Te zabawki należą do [MASK].',
    'Policjant przygląda się [MASK].',
    'Na środku skrzyżowania widać [MASK].',
    'Właściciel samochodu widział złodzieja z [MASK].',
    'Prezydent z premierem rozmawiali wczoraj o [MASK].',
    'Witaj drogi [MASK].',
]

In [7]:
def predict_gap(sentence, pipeline):
    input_sentence = sentence.replace('[MASK]', pipeline.tokenizer.mask_token)
    predictions = pipeline(input_sentence)
    results = [sentence.replace('[MASK]', pred['token_str']) for pred in predictions]
    results = [result.replace('</w>', '') for result in results]
    return results

In [8]:
def predict_gaps_in_sentences(sentences, pipelines, models_names):
    for sentence in sentences:
        print(f'\033[1m\'{sentence}\':\033[0m')
        for pipeline, model_name in zip(pipelines, models_names):
            print(f'\033[1m   {model_name}:\033[0m')
            for result in predict_gap(sentence, pipeline):
                print(f'     {result}')
            print()
    print()

In [9]:
predict_gaps_in_sentences(sentences, pipelines, models_names)

'Warszawa to największe [MASK].':
   DKleczek:
     Warszawa to największe miasto.
     Warszawa to największe województwo.
     Warszawa to największe lotnisko.
     Warszawa to największe miasteczko.
     Warszawa to największe państwo.

   Geotrend:
     Warszawa to największe miasto.
     Warszawa to największe miasta.
     Warszawa to największe woj.
     Warszawa to największe Warszawa.
     Warszawa to największe miast.

   Allegro Herbert:
     Warszawa to największe miasto.
     Warszawa to największe miasta.
     Warszawa to największe ..
     Warszawa to największe lotnisko.
     Warszawa to największe miast.

'Te zabawki należą do [MASK].':
   DKleczek:
     Te zabawki należą do ciebie.
     Te zabawki należą do mnie.
     Te zabawki należą do nas.
     Te zabawki należą do pana.
     Te zabawki należą do niego.

   Geotrend:
     Te zabawki należą do tzw.
     Te zabawki należą do pt.
     Te zabawki należą do pl.
     Te zabawki należą do odc.
     Te zabawki należą do ok

# Zadanie 4

Predykcja brakujących słów w zdaniach nastawiona na poprawny rodziaj

In [10]:
sentences_2 = [
    'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].',
    'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].',
]

In [11]:
predict_gaps_in_sentences(sentences_2, pipelines, models_names)

'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].':
   DKleczek:
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgodził.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie martwił.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił.

   Geotrend:
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##wiedział.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie było.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie stało.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dził.
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dzi.

   Allegro Herbert:
     Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał.
     Gdybym wiedzia

# Zadanie 5

Predykcja brakujących słów w zdaniach nastawiona na logicznie dopasowanie do kontesktu

In [12]:
sentences_3 = [
    '[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.',
    'W wakacje odwiedziłem [MASK], który jest stolicą Islandii.',
    'Informatyka na [MASK] należy do najlepszych kierunków w Polsce.',    
]

In [13]:
predict_gaps_in_sentences(sentences_3, pipelines, models_names)

'[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.':
   DKleczek:
     Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Mięso wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Piwo wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.

   Geotrend:
     Jego wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Za wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Po wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
     Ich wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.

   Allegro Herbert:
     Woda 

# Zadanie 6

1. Which of the models produced the best results?

Wyraźnie gorsze wyniki od pozostałych dwóch modeli uzyskał Geotrend. Jedynym przypadkiem w którym zwrócił lepszy wynik było podanie Reykjaviku jako stolicy Isnadii. DKleczek i Herbert zwróciły wyniki na bardzo podobnym poziomie. Pod względem odmiany przez przypadki oba modele zadziałały bardzo dobrze, podobnie pod względem wykrywania poprawnego rodzaju czasowników. Predykcja logicznych słów obu modelom poszła przeciętnie. Herbert wyraźnie lepiej poradził sobie z trzecim zdaniem ('Informatyka na [MASK] należy do najlepszych kierunków w Polsce.'), więc ostatecznie powiedziałbym, że najlepsze wyniki pozwolił uzyskać model Allegro Herbert.

2. Was any of the models able to capture Polish grammar?

DKleczek i Herbert bardzo dobrze poradziły sobie z predykcją słów pasujących do kontekstu pod względem gramatycznym. W zdaniach celujących w testowanie logicznych odpowiedzi modeli również wszystkie predykcje DKleczka i Herberta były poprawne gramatycznie. Geotrend tylko w pojedynczych przypadkach przewidywał słowa w poprawnym przypadku czy rodzaju.

3. Was any of the models able to capture long-distant relationships between the words?

Najlepiej to widać analizując rezultaty dla zdań:
- 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].',
- 'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].',
- 'Prezydent z premierem rozmawiali wczoraj o [MASK].'
Dla dwóch pierwszych DKleczek i Herbert zwróciły bardzo dobre wyniki pasujące gramatycznie i tematycznie (z przymrużeniem oka patrząc na odpowiedzi z zabijaniem). Geotrend nie był w stanie wykryć odpowiednich słów, ani pasujących gramatycznie, ani logicznie. Dla trzeciego zdania DKleczek i Herbert zwróciły poprawne gramatycznie słowa. Ponadto DKleczek zwrócił słowa 'Polsce', 'budżecie', 'ustawie', które bardzo pasowały tematycznie. Herbert również przewidział nieźle pasujące tematycznie słowa jak 'sprawie', 'sytuacji', 'prywatyzacji'. Geotrend dla ostatniego zdania znów poradził sobie najgorzej, 3/5 wyników było poprawnych gramatycznie, ale niezbyt związanych z kontekstem zdania.

4. Was any of the models able to capture world knowledge?

- '[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.'
  - DKleczek i Herbert przewidziały wodę, pojawiło się też piwo, krew, więc dość logiczne odpowiedzi
  - Geotreng totalnie sobie nie poradził
- 'W wakacje odwiedziłem [MASK], który jest stolicą Islandii.'
  - Herbert dopasował się gramatycznie i kontekstowo zwracając nazwy miast, niestety wszystkie błędne
  - DKleczek dopasował się gramatycznie, niestety zwracał nazwy państw, dodatkowo zwrócił słowo 'kraj', próbując wybrnąć z sytuacji nie używając wiedzy o świecie, ale znów powinien był zwrócić słowo 'miasto'
  - Geotrend poradził sobie najlepiej, bo udało mu się zaproponowć Reykjvik i słowo miasto
- 'Informatyka na [MASK] należy do najlepszych kierunków w Polsce.'
  - DKleczek i Geotrend zwróciły poprawne gramatycznie, ale nielogiczne odpowiedzi
  - Herbert poradził sobie bardzo dobrze zwracając nazwy różnych uczelni, udało mu się nawet trafić w jedyną poprawną odpowiedź 'Informatyka na AGH należy do najlepszych kierunków w Polsce.'

5. What are the most striking errors made by the models?

Najpopularniejszymi błędami są zupełnie niepoprawnie przewidziane przez Geotrend słowa (często skróty). Większość najbardziej wyraźnych błędów w modelach DKleczek i Herbert pojawia się gdy modelowi nie udaje się dopasować do wiedzy o świecie: 'Słońce/Ogień wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.', 'W wakacje odwiedziłem Meksyk, który jest stolicą Islandii.', 'Informatyka na wsi należy do najlepszych kierunków w Polsce.'. Błędy gramatyczne nie zwracają na siebię aż tak bardzo uwagi. Popularne w rezultatach zwracanych przez Geotreng są również błędy zupełnie absurdalne jak np. 'Na środku skrzyżowania widać rok.'.
